In [2]:
from utils_real_data import *
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
# LOAD THE DATA

mainz_dict, trans_dict, vdx_dict = np.load('Data/mainz_dict.npy', allow_pickle=1).item(), np.load('Data/transbig_dict.npy', allow_pickle=1).item(), np.load('Data/vdx_dict.npy', allow_pickle=1).item()

mainz_3, mainz_pam, mainz_all = np.asarray(mainz_dict['HK_3'].values[:, 1:].T, dtype = float) , np.asarray(mainz_dict['PAM50'].values[:,1:].T, dtype = float), np.asarray(mainz_dict['all'].values[:,1:].T, dtype = float)
trans_3, trans_pam, trans_all = np.asarray(trans_dict['HK_3'].values[:, 1:].T, dtype = float) , np.asarray(trans_dict['PAM50'].values[:,1:].T, dtype = float), np.asarray(trans_dict['all'].values[:,1:].T, dtype = float)
vdx_3, vdx_pam, vdx_all = np.asarray(vdx_dict['HK_3'].values[:, 1:].T, dtype = float) , np.asarray(vdx_dict['PAM50'].values[:,1:].T, dtype = float), np.asarray(vdx_dict['all'].values[:,1:].T, dtype = float)

In [7]:
### PCA on PAM 50

pca = PCA()
dd_ = [mainz_pam, trans_pam, vdx_pam]
pam_pca = pca.fit(np.concatenate([dd_[0], dd_[1], dd_[2]])) # create common matrix

# plt.plot(range(1,len(pam_pca.explained_variance_ratio_)+1), pam_pca.explained_variance_ratio_.cumsum())
# plt.hlines(y=.75, xmin = 1, xmax = pam_pca.n_features_)
# plt.vlines(ymin = 0, ymax = 1, x = 15) # retain 15 components
# plt.show()

pam_pca  = PCA(n_components=15)
pam_pca_ = pam_pca.fit_transform(np.concatenate([dd_[0], dd_[1], dd_[2]]))
mainz_pam_pca, trans_pam_pca, vdx_pam_pca = pam_pca_[:len(mainz_pam)], pam_pca_[len(mainz_pam):len(mainz_pam)+len(trans_pam)], pam_pca_[len(mainz_pam)+len(trans_pam):]

In [8]:
# PCA on 1% most variable genes

threshold = int(trans_all.shape[1]/200)

variation_mainz, variation_trans, variation_vdx = sp.stats.variation(mainz_all, axis = 0), sp.stats.variation(trans_all, axis = 0), sp.stats.variation(vdx_all, axis = 0) # compute per gene variation
index_mainz, index_trans, index_vdx = set(np.argsort(variation_mainz)[::-1][:threshold]), set(np.argsort(variation_trans)[::-1][:threshold]), set(np.argsort(variation_vdx)[::-1][:threshold]) # compute top 1% genes in each dataset
variable_genes_index = list(index_mainz.intersection(index_trans, index_vdx)) # compute genes whihc are in top 1% most variable genes in all 3 datasets
mainz_top, trans_top, vdx_top = mainz_all[:, variable_genes_index], trans_all[:, variable_genes_index], vdx_all[:,variable_genes_index] # compute variable genes

pca = PCA()
dd_ = [mainz_top, trans_top, vdx_top]
top_pca = pca.fit(np.concatenate([dd_[0], dd_[1], dd_[2]])) # create common matrix

# plt.plot(range(1,len(top_pca.explained_variance_ratio_)+1), top_pca.explained_variance_ratio_.cumsum())
# plt.hlines(y=.75, xmin = 1, xmax = top_pca.n_features_)
# plt.vlines(ymin = 0, ymax = 1, x = 30) # retain 30 components
# plt.show()

top_pca  = PCA(n_components=30)
top_pca_ = top_pca.fit_transform(np.concatenate([dd_[0], dd_[1], dd_[2]]))
mainz_top_pca, trans_top_pca, vdx_top_pca = top_pca_[:len(mainz_top)], top_pca_[len(mainz_top):len(mainz_top)+len(trans_top)], top_pca_[len(mainz_top)+len(trans_top):]

In [ ]:
methods_list = ['Kmeans', 'Birch', 'AgglomerativeClustering']
real_data = {}
min_num_cl, max_num_cl, num_boots = 3, 8, 50

datasets_dic = {}
datasets_dic['PAM_50'], datasets_dic['HK_3'], datasets_dic['PAM_50_PCA'], datasets_dic['top'], datasets_dic['top_pca'] = [mainz_pam, trans_pam, vdx_pam], [mainz_3, trans_3, vdx_3], [mainz_pam_pca, trans_pam_pca, vdx_pam_pca], [mainz_top, trans_top, vdx_top], [mainz_top_pca, trans_top_pca, vdx_top_pca]

which_ds = ['mainz', 'trans', 'vdx']

for data_ in ['PAM_50_PCA', 'top', 'top_pca']:
    datasets, real_data[data_] = datasets_dic[data_], {}
    print('Using ', data_)
    for method in methods_list:
        print('\tEvaluating replicability with ', method)
        real_data[data_][method] = {}
        real_data[data_][method]['ari'], real_data[data_][method]['ami'], real_data[data_][method]['true_labels'], real_data[data_][method]['predicted_labels'] = {}, {}, {}, {}
        for i, train_set in tqdm_notebook(enumerate(datasets)):
            print('\t\tTraining on ', which_ds[i])
            for j, test_set in enumerate(datasets):
                print('\t\t\tTesting on ', which_ds[j],'; for k in 2,...,', max_num_cl)
                test_set = datasets[j]
                
                ari, ami, true_labs, predicted_labels = run_exercise_1(train_set, test_set, method, num_boots, max_num_cl, AFFINITY, LINKAGE)
                real_data[data_][method]['ari'][which_ds[i]+'_'+which_ds[j]], real_data[data_][method]['ami'][which_ds[i]+'_'+which_ds[j]] = ari, ami
                real_data[data_][method]['true_labels'][which_ds[i]+'_'+which_ds[j]], real_data[data_][method]['predicted_labels'][which_ds[i]+'_'+which_ds[j]] = true_labs, predicted_labels
#     np.save('Results/scores_labels', real_data)

Using  PAM_50_PCA
	Evaluating replicability with  Kmeans


<ipython-input-13-4a09e85aa900>:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, train_set in tqdm_notebook(enumerate(datasets)):


0it [00:00, ?it/s]

		Training on  mainz
			Testing on  mainz ; for k in 2,..., 8


  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

			Testing on  trans ; for k in 2,..., 8


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]